In [2]:
from IPython.display import display, HTML

display(HTML("""
    <link rel="stylesheet" href="/style.css">
"""))


In [8]:
from monarch import Hatch
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import pathlib
import monarch.metamorphoses as meta
from monarch.utils import get_valve_events



# Interactive Monarch Demonstration
The first graph demonstrates a simple interactive PV loop visualization using the Monarch cardiac model. The
second displays the Geometry of the heart created by the Monarch model.

In [9]:
input_dir = pathlib.Path.cwd() / 'input_files'
input_human = input_dir / 'human'

In [10]:
def interactive_pv_loop(hr=60, sbv=1500, contractility=0.15):
    beat_mod = Hatch(input_human)
    beat_mod.change_pars({
        "HR": hr,
        "sbv": sbv,
    })
    beat_mod.heart.sf_act[0] = contractility  # LFW contractility
    beat_mod.just_beat_it()



In [5]:
### Extended: Interactive Geometry + PV Loop with Stiffness and Wall Volume
import ipywidgets as widgets
import numpy as np

def interactive_pv_loop_extended(hr=60, sbv=1500, lfw_contractility=0.15,rfw_contractility=0.10, septal_contractility=0.15, stiffness=0.017, wall_volume=40000):
    beat_mod = Hatch(input_human)
    beat_mod.change_pars({
        "HR": hr,
        "sbv": sbv
    })

    # Set contractility
    beat_mod.heart.sf_act[0] = lfw_contractility   # LFW
    beat_mod.heart.sf_act[1] = rfw_contractility
    beat_mod.heart.sf_act[2] = septal_contractility  # Septal Wall

    # Set wall volume
    beat_mod.heart.vw[0] = wall_volume  # LFW volume

    # Set stiffness parameters
    #beat_mod.heart.c_3 = np.full_like(beat_mod.heart.c_3, stiffness)
    # only change c4 not c3
    beat_mod.heart.c_4 = np.full_like(beat_mod.heart.c_4, stiffness * 600)

    # Run simulation
    beat_mod.just_beat_it()

    #fig = plt.figure(figsize=(14, 6))
    # PV Loop
    meta.pv_loop(beat_mod, compartments=("LV", "RV"))
    #ax1 = fig.add_subplot(1, 2, 1)
    #ax1.set_title(f"Heart Rate: {hr} bpm | Stressed Blood Volume: {sbv} mL")
    #meta.pv_loop(beat_mod, compartments=("LV", "RV"))

    # Geometry
    meta.cardiac_geometry(beat_mod, real_wall_thickness=True)
    #ax1.set_title(f"Geometry (Vol: {wall_volume} mm³, Stiffness: {stiffness:.3f})")

    #plt.tight_layout()
    #plt.show()


# — Global parameters —
hr_slider = widgets.IntSlider(
    value=60,
    min=30,
    max=180,
    step=1,
    description="HR (bpm):"
)

sbv_slider = widgets.IntSlider(
    value=1500,
    min=200,
    max=3000,
    step=50,
    description="SBV (mL):"
)

# — Left Free Wall (LFW) contractility —
lfw_contractility_slider = widgets.FloatSlider(
    value=0.15,
    min=0.05,
    max=0.30,
    step=0.01,
    description="LFW Contractility"
)

# — Septal contractility (shared between walls) —
septal_contractility_slider = widgets.FloatSlider(
    value=0.15,
    min=0.05,
    max=0.30,
    step=0.01,
    description="Septal Contractility"
)

# — Right Free Wall (RFW) contractility —
rfw_contractility_slider = widgets.FloatSlider(
    value=0.10,
    min=0.02,
    max=0.25,
    step=0.01,
    description="RFW Contractility"
)

# — Wall stiffness (applies to whole heart; c4 multiplier) —
stiffness_slider = widgets.FloatSlider(
    value=0.017,
    min=0.001,
    max=0.020,
    step=0.001,
    description="Stiffness (c4)"
)

# — Wall volume (LFW volume, index 0) —
wall_volume_slider = widgets.IntSlider(
    value=40000,
    min=10000,
    max=80000,
    step=1000,
    description="Wall Volume (µL)"
)

# — Vascular capacitance (new) —
capacitance_slider = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=5.0,
    step=0.1,
    description="Capacitance (C)"
)

# — Vascular resistance (new) —
resistance_slider = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=10.0,
    step=0.1,
    description="Resistance (R)"
)


global_box = widgets.VBox([
    widgets.HTML("<b>Global Parameters</b>"),
    hr_slider,
    sbv_slider
])

# 3.2. Left Free Wall & Septal Wall
lfw_box = widgets.VBox([
    widgets.HTML("<b>Left Free Wall (LFW)</b>"),
    lfw_contractility_slider,
    septal_contractility_slider,
])

rfw_box = widgets.VBox([
    widgets.HTML("<b>Right Free Wall (RFW)</b>"),
    rfw_contractility_slider
])

shared_mech_box = widgets.VBox([
    widgets.HTML("<b>Myocardial Mechanics </b>"),
    stiffness_slider,
    wall_volume_slider
])

# 3.5. Electrical parameters (capacitance & resistance)
vasculature_box = widgets.VBox([
    widgets.HTML("<b>Electrical Parameters</b>"),
    capacitance_slider,
    resistance_slider
])


ui = widgets.VBox([
    global_box,
    widgets.HBox([lfw_box, rfw_box]),
    shared_mech_box,
    vasculature_box
])


out = widgets.interactive_output(
    interactive_pv_loop_extended,
    {
        "hr": hr_slider,
        "sbv": sbv_slider,
        "lfw_contractility": lfw_contractility_slider,
        "septal_contractility": septal_contractility_slider,
        "rfw_contractility": rfw_contractility_slider,
        "stiffness": stiffness_slider,
        "wall_volume": wall_volume_slider
    }
)

display(ui, out)




Output()

The below graph demonstrates a Wiggers diagram showing different pressures at different times of the cardiac cycle.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interactive, fixed, FloatSlider

beat = Hatch(input_human)
beat.just_beat_it()

def wiggers_diagram_with_dot(model, t_dot, cmap="cubehelix",
                             show_fig=True, file_type="pdf",
                             file_path=None, file_name="wiggers",
                             fig_size=(6.4, 4.8)):
    """
    Plot the Wiggers diagram for `model` (after simulation) and place a red dot at time = t_dot.
    Assumes:
      - model.time       is a 1D array of time points
      - model.pressures  is a 2D array whose columns are [RA, LA, LV, Aorta]
                           (we index LV = [:,2], LA = [:,1], Aorta = [:,3])
      - model.volumes    is a 2D array whose LV volume is at [:,2]
      - get_valve_events(model) returns a dict of indices for mv_closes, av_opens, av_closes, mv_opens
      - finish_plot(...) will either show or save the figure
    """
    cmap_vals = sns.color_palette(cmap, 4, as_cmap=False)

    sns.set_theme(style="white")
    fig, ax = plt.subplots(
        3, 1,
        figsize=fig_size,
        gridspec_kw={'height_ratios': [3, 1.5, 0.2]}
    )

    ax[0].plot(model.time, model.pressures[:, 2], color=cmap_vals[0], linewidth=3, label="LV")
    ax[0].plot(model.time, model.pressures[:, 1], color=cmap_vals[1], linewidth=3, label="LA")
    ax[0].plot(model.time, model.pressures[:, 3], color=cmap_vals[2], linewidth=3, label="Aortic")
    ax[0].set_ylim(bottom=0)

    ax[1].plot(model.time, model.volumes[:, 2], color=cmap_vals[3], linewidth=3, label="LV Volume")

    time_events = get_valve_events(model)
    mv_color = "#888888"
    av_color = "#555555"
    for axi in ax:
        axi.axvline(
            x=model.time[time_events["mv_closes"]],
            c=mv_color, linestyle="--", ymin=-0.5, ymax=1.0,
            linewidth=2, zorder=0, clip_on=False
        )
        axi.axvline(
            x=model.time[time_events["av_opens"]],
            c=av_color, linestyle=":", ymin=-0.5, ymax=1.0,
            linewidth=2, zorder=0, clip_on=False
        )
        axi.axvline(
            x=model.time[time_events["av_closes"]],
            c=av_color, linestyle="--", ymin=-0.5, ymax=1.0,
            linewidth=2, zorder=0, clip_on=False
        )
        axi.axvline(
            x=model.time[time_events["mv_opens"]],
            c=mv_color, linestyle=":", ymin=-0.5, ymax=1.0,
            linewidth=2, zorder=0, clip_on=False
        )

    ax[0].annotate(
        "Ventricular pressure",
        xy=(1.025 * model.time[-1], model.pressures[-1, 2] - 3),
        fontsize=12, ha="left", annotation_clip=False, color=cmap_vals[0]
    )
    ax[0].annotate(
        "Atrial pressure",
        xy=(1.025 * model.time[-1], model.pressures[-1, 1] + 3),
        fontsize=12, ha="left", annotation_clip=False, color=cmap_vals[1]
    )
    ax[0].annotate(
        "Aortic pressure",
        xy=(1.025 * model.time[-1], model.pressures[-1, 3]),
        fontsize=12, ha="left", annotation_clip=False, color=cmap_vals[2]
    )
    ax[1].annotate(
        "Ventricular volume",
        xy=(1.025 * model.time[-1], model.volumes[-1, 2]),
        fontsize=12, ha="left", annotation_clip=False, color=cmap_vals[3]
    )

    top_ylim = ax[0].get_ylim()[1]
    ax[0].annotate(
        "MV closes",
        xy=(model.time[time_events["mv_closes"]], top_ylim),
        fontsize=12, ha="left", annotation_clip=False,
        color=mv_color, rotation=45
    )
    ax[0].annotate(
        "AV opens",
        xy=(model.time[time_events["av_opens"]], top_ylim),
        fontsize=12, ha="left", annotation_clip=False,
        color=av_color, rotation=45
    )
    ax[0].annotate(
        "AV closes",
        xy=(model.time[time_events["av_closes"]], top_ylim),
        fontsize=12, ha="left", annotation_clip=False,
        color=av_color, rotation=45
    )
    ax[0].annotate(
        "MV opens",
        xy=(model.time[time_events["mv_opens"]], top_ylim),
        fontsize=12, ha="left", annotation_clip=False,
        color=mv_color, rotation=45
    )

    for i in range(3):
        ax[i].spines['right'].set_visible(False)
        ax[i].spines['top'].set_visible(False)
        ax[i].spines['bottom'].set_visible(False)
        ax[i].xaxis.set_visible(False)
        ax[i].set_xlim(model.time[0], model.time[-1])

    ax[2].spines['left'].set_visible(False)
    ax[2].yaxis.set_visible(False)
    ax[2].spines['bottom'].set_visible(True)
    ax[2].xaxis.set_visible(True)

    p_lv_dot = np.interp(t_dot, model.time, model.pressures[:, 2])
    v_lv_dot = np.interp(t_dot, model.time, model.volumes[:, 2])
    ax[0].plot(t_dot, p_lv_dot, 'ro', markersize=8)
    ax[1].plot(t_dot, v_lv_dot, 'ro', markersize=8)

    ax[0].set_ylabel("Pressure (mmHg)", fontsize=12)
    ax[1].set_xlabel("Time (s)", fontsize=12)
    ax[1].set_ylabel("Volume (mL)", fontsize=12)

    plt.tight_layout()
    meta.finish_plot(fig, file_path, file_name, file_type, show_fig)

time_slider = FloatSlider(
    value=beat.time[0],
    min=float(beat.time[0]),
    max=float(beat.time[-1]),
    step=float(beat.time[1] - beat.time[0]),
    description="Time (s)",
    continuous_update=True,
    style={'description_width': 'initial'}
)

#
interactive(
    wiggers_diagram_with_dot,
    model=fixed(beat),
    t_dot=time_slider,
    cmap=fixed("cubehelix"),
    show_fig=fixed(True),
    file_type=fixed("pdf"),
    file_path=fixed(None),
    file_name=fixed("wiggers"),
    fig_size=fixed((6.4, 4.8))
)
